In [ ]:
from flytekit.configuration import set_flyte_config_file, platform
set_flyte_config_file("../../notebook.config")
print("Connected to {}".format(platform.URL.get()))

from jupyter_helpers import print_console_url

# Section I: Execute registered Tasks

## Option 1: Execute without having the right environment setup
This option involves fetching the remote task and then requesting an execution in your own namespace.
This concept is what is also used in Sharing.

To start an execution following pieces of information are required
 - project: The project where the task was registered
 - domain: Domain where the task was registered
 - name: Name of the registered task. This is the canonical name (usually includes the module name). You can lookup using flyte-cli or console or arrive at it using "root_module.module....task" pattern
 - Version! This is the version of the registered task
 
**Note** Version is the most important to pinpoint the task to execute. At the same time Flyte can have multiple versions active. Cookbook build system uses Git-SHA to version a release. So you can use the latest git-sha from flytesnacks as the release identifier.


In [ ]:
from flytekit.common.tasks.task import SdkTask
version = "f6268cf16c52bb457b3a3404e98e68b307bad834" ## Replace this
remote_task = SdkTask.fetch(project="flytesnacks", domain="development", name="task.tasks.scale", version=version)

In [ ]:
exe = remote_task.launch(project="flytesnacks", domain="development", inputs={"image":"https://miro.medium.com/max/1400/1*qL8UYfaStcEo_YVPrA4cbA.png", "scale_factor":4})
print_console_url(exe)

### Load outputs for introspection
**TODO** This does not work on sandbox environment today as minio is not available on the same port

In [ ]:
exe.wait_for_completion()
# In sandbox outputs does not work because the dns points to k8s local address, but this is not available on localhost
#. we can create a hostfile entry but that would be too complicated
exe.outputs

## Option 2: Execute by loading the current code 

This is only supported for non-container (like queries etc), pre-built containers like Sagemaker etc and Raw containers tasks. For tasks that are written in containers, this is not directly supported as it would easily confuse the user about false iteration, without building a container.

**NOTE** this is an active area of work, come back again and you will see an update.


# Section II: Execute registered Workflows

## Option 1: Execute without the environment setup
This option involves fetching the remote workflow (pre-registered) and then requesting an execution in your own namespace.
This concept is what is also used in Sharing.

Flyte Workflows are not directly executable. Only launch plans are executable. LaunchPlans allow partial specialization of inputs and allow specifying schedules. This enables Flyte workflows to have multiple schedules at the same time (with different default input values). Refer to the docs to understand LaunchPlans in more depth.

**NOTE** Flytekit automatically creates a **default** launch plan when a workflow is registered. A Default launchplan is exactly like a Workflow.

To start an execution following pieces of information are required
 - project: The project where the task was registered
 - domain: Domain where the task was registered
 - name: Name of the registered Launch plan - *for default launch plans the name is identical to the workflow name*. The Workflow name is the canonical name (usually includes the module name). You can lookup using flyte-cli or console or arrive at it using "root_module.module....task" pattern
 - Version! This is the version of the registered task
 
**Note** Version is the most important to pinpoint the workflow/launchplan to execute. At the same time Flyte can have multiple versions active. Cookbook build system uses Git-SHA to version a release. So you can use the latest git-sha from flytesnacks as the release identifier.


In [ ]:
from flytekit.common.launch_plan import SdkLaunchPlan
remote_lp = SdkLaunchPlan.fetch(project="flytesnacks", domain="development", name="workflows.workflows.FailingWorkflow", version=version)

In [ ]:
exe = remote_lp.execute(project="flytesnacks", domain="development", inputs={"in_image": "https://miro.medium.com/max/1400/1*qL8UYfaStcEo_YVPrA4cbA.png", "angle": 180.0})
print_console_url(exe)

# Section III: Load Error for introspection
Flyte returns the error captured during execution through the API.
This can be captured in multiple ways, if using flytekit, then "raising" any Python Exception will be captured. If using raw container, then raw stdout / stderr will be captured, or error can be written to a special file.

In [ ]:
from flytekit.common.workflow_execution import SdkWorkflowExecution
exe.wait_for_completion()
name = exe.id.name
# WE are using the name to fetch the execution. This is a contrived example and only exists to show how this can be done
exe = SdkWorkflowExecution.fetch(project="flytesnacks", domain="development", name=name)
exe.error

# Section IV: Relaunch the failed execution
Sometimes relaunching an execution is what is required just with a different parameter. This is usually preferable if one of the inputs was incorrect or if the workflow execution failed due to a system or any other retryable failure.

### Lets fetch the execution information first


In [ ]:
from flytekit.common.workflow_execution import SdkWorkflowExecution
exe = SdkWorkflowExecution.fetch(project="flytesnacks", domain="development", name=name)


### Now fetch the launch plan to execute, note this can be different version.

In [ ]:
from flytekit.common.launch_plan import SdkLaunchPlan
remote_lp = SdkLaunchPlan.fetch(project="flytesnacks", domain="development", name="workflows.workflows.FailingWorkflow", version=version)
## Now just execute like previous